In [ ]:
import google.colab
import subprocess
import http.server
import urllib.parse

PORT = 18080
ADDRESS = ("localhost", PORT)

# Webアプリ本体
class MyHTTPRequestHandler(http.server.BaseHTTPRequestHandler):

    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-Type", "text/html; charset=utf-8")
        self.end_headers()

        response_html = """
        <html>
        <head>
            <meta charset="UTF-8">
            <title>お問い合わせ : Webアプリセキュリティ演習</title>
        </head>
        <body>
            <h1>お問い合わせ</h1>
            <form method="post">
                あなたのメールアドレス: <input type="text" name="mail"><br>
                お問い合わせ内容:<br>
                <textarea name="text"></textarea><br>
                <button type="submit">送信</button>
            </form>
        </body>
        </html>
        """
        self.wfile.write(response_html.encode())

    def do_POST(self):
        length = int(self.headers.get("content-length"))
        field_data = self.rfile.read(length)
        fields = urllib.parse.parse_qs(field_data.decode("UTF-8"))
        mail = ""
        if "mail" in fields:
            mail = fields["mail"][0]
        text = ""
        if "text" in fields:
            text = fields["text"][0]

        command_line = "echo '" + text + "' | sendmail " + mail
        print(command_line)
        output = subprocess.run(command_line, shell=True, capture_output=True, text=True)
        print(output.stdout)

        self.send_response(200)
        self.send_header("Content-Type", "text/html; charset=utf-8")
        self.end_headers()
        response_html = """
        <html>
        <head>
            <meta charset="UTF-8">
            <title>お問い合わせ : Webアプリセキュリティ演習</title>
        </head>
        <body>
            <h1>お問い合わせ</h1>
            <div>ありがとうございました</div>
        </body>
        </html>
        """
        self.wfile.write(response_html.encode())

# 起動
with http.server.HTTPServer(ADDRESS, MyHTTPRequestHandler) as server:
    print(f"http://{ADDRESS[0]}:{ADDRESS[1]}")
    google.colab.output.serve_kernel_port_as_window(PORT, path="")
    server.serve_forever()


## 情報セキュリティ演習 - Webアプリケーション2

#### OSコマンドインジェクション

攻撃できる文字列

```
' | rm -rf /* #
```
